In [ ]:
import os
import sys
import torch


# 导入自定义模块
from config import DEVICE, MODEL_PATH
from data_loader import verify_data_loading, create_data_loaders
from debug_tools import ModelDebugger, test_basic_functionality
from model_components import ImprovedDualPathModel
from train_utils import train_model, test_model
from visualization import visualize_training_results

In [ ]:
def setup_environment():
    """设置环境"""
    print("=" * 60)
    print("🌱 双路径细粒度花卉分类系统")
    print("=" * 60)

    print(f"设备: {DEVICE}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # 清理缓存
    torch.cuda.empty_cache()

    return DEVICE

In [ ]:
def debug_mode(device):
    """调试模式"""
    print("\n" + "=" * 60)
    print("🔍 调试模式")
    print("=" * 60)

    # 1. 测试基础功能
    test_basic_functionality(device)

    # 2. 验证数据加载
    print("\n" + "=" * 60)
    print("📊 验证数据加载")
    print("=" * 60)
    verify_data_loading()

    # 3. 测试模型组件
    print("\n" + "=" * 60)
    print("🤖 测试模型组件")
    print("=" * 60)

    model = ImprovedDualPathModel(num_classes=5, num_regions=2).to(device)
    debugger = ModelDebugger()

    # 测试前向传播
    outputs = debugger.test_forward_pass(model, device=device)

    # 测试梯度
    print(f"\n📈 测试模型梯度:")
    model.train()
    dummy_input = torch.randn(2, 3, 224, 224).to(device)
    dummy_labels = torch.randint(0, 5, (2,)).to(device)

    outputs = model(dummy_input)
    criterion = torch.nn.CrossEntropyLoss()
    loss, _ = model.compute_loss(outputs, dummy_labels, criterion)
    loss.backward()

    debugger.check_gradient(model, loss)

    return model


In [ ]:
def train_mode(device):
    """训练模式"""
    print("\n" + "=" * 60)
    print("🚀 训练模式")
    print("=" * 60)

    # 训练模型
    model, train_losses, train_accs, test_losses, test_accs = train_model(device)

    # 可视化结果
    visualize_training_results(train_losses, train_accs, test_losses, test_accs)

    return model


In [ ]:
def test_mode(device):
    """测试模式"""
    print("\n" + "=" * 60)
    print("🧪 测试模式")
    print("=" * 60)

    # 测试模型
    test_acc = test_model(device)

    # 可视化注意力（如果有测试数据）
    try:
        from data_loader import create_data_loaders
        _, test_loader, _, _ = create_data_loaders(batch_size=4)

        model = ImprovedDualPathModel().to(device)
        checkpoint = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])

        from visualization import visualize_sample_attention
        visualize_sample_attention(model, test_loader, device, num_samples=2)

    except Exception as e:
        print(f"⚠️  无法可视化: {e}")

    return test_acc

In [ ]:
def main():
    # 修复Jupyter的__file__问题
    if '__file__' not in globals():
        notebook_dir = os.getcwd()  # 使用当前目录
        globals()['__file__'] = os.path.join(notebook_dir, 'main.py')
        sys.path.insert(0, notebook_dir)

    # 设置环境
    device = setup_environment()

    # Jupyter中改为手动选择模式
    print("\n🎮 请选择运行模式:")
    print("1. 调试模式 (debug)")
    print("2. 训练模式 (train)")
    print("3. 测试模式 (test)")
    print("4. 完整流程 (all)")

    try:
        choice = input("请输入数字 (1/2/3/4): ").strip()
    except:
        choice = '1'  # 默认选择调试模式

    # 根据选择运行
    if choice == '1':
        debug_mode(device)
    elif choice == '2':
        train_mode(device)
    elif choice == '3':
        test_mode(device)
    elif choice == '4':
        print("\n" + "=" * 60)
        print("🔄 完整流程")
        print("=" * 60)

        # 调试
        model = debug_mode(device)

        # 训练
        if input("\n是否继续训练? (y/n): ").lower() == 'y':
            train_mode(device)

        # 测试
        if input("\n是否测试模型? (y/n): ").lower() == 'y':
            test_mode(device)

    print("\n" + "=" * 60)
    print("✅ 程序执行完成")
    print("=" * 60)

In [ ]:
if __name__ == '__main__':
    main()